In [1]:
import mne

### Import Data

In [2]:
# https://physionet.org/content/eegmmidb/1.0.0/#files-panel
raw_data = mne.io.read_raw_edf('../Datasets/physio_mi/S001/S001R04.edf')

Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [3]:
raw_data.annotations

<Annotations | 30 segments: T0 (15), T1 (8), T2 (7)>

In [4]:
#Event 0 corresponds to rest

#3,4,7,8,11,12 run : Event 1 corresponds left fist 
#5,6,9,10,13,14 run : Event 1 corresponds both fists

#3,4,7,8,11 run : Event 2 corresponds right fist 
#5,6,9,10,13 run : Event 2 corresponds both feet

#first event 672 => 672/160hz = 4.2sec events

events, event_id = mne.events_from_annotations(raw_data)
events

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]


array([[    0,     0,     1],
       [  672,     0,     3],
       [ 1328,     0,     1],
       [ 2000,     0,     2],
       [ 2656,     0,     1],
       [ 3328,     0,     2],
       [ 3984,     0,     1],
       [ 4656,     0,     3],
       [ 5312,     0,     1],
       [ 5984,     0,     3],
       [ 6640,     0,     1],
       [ 7312,     0,     2],
       [ 7968,     0,     1],
       [ 8640,     0,     3],
       [ 9296,     0,     1],
       [ 9968,     0,     2],
       [10624,     0,     1],
       [11296,     0,     3],
       [11952,     0,     1],
       [12624,     0,     2],
       [13280,     0,     1],
       [13952,     0,     2],
       [14608,     0,     1],
       [15280,     0,     3],
       [15936,     0,     1],
       [16608,     0,     2],
       [17264,     0,     1],
       [17936,     0,     3],
       [18592,     0,     1],
       [19264,     0,     2]])

In [62]:
#raw2
raw_data2 = mne.io.read_raw_edf('../Datasets/physio_mi/S104/S104R08.edf')

Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S104/S104R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [63]:
raw_data2.annotations

<Annotations | 26 segments: T0 (13), T1 (7), T2 (6)>

In [64]:
events, event_id = mne.events_from_annotations(raw_data2)
events

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]


array([[    0,     0,     1],
       [  656,     0,     2],
       [ 1312,     0,     1],
       [ 1968,     0,     3],
       [ 2624,     0,     1],
       [ 3280,     0,     3],
       [ 3936,     0,     1],
       [ 4592,     0,     2],
       [ 5248,     0,     1],
       [ 5904,     0,     2],
       [ 6560,     0,     1],
       [ 7216,     0,     3],
       [ 7872,     0,     1],
       [ 8528,     0,     2],
       [ 9184,     0,     1],
       [ 9840,     0,     3],
       [10496,     0,     1],
       [11152,     0,     2],
       [11808,     0,     1],
       [12464,     0,     3],
       [13120,     0,     1],
       [13776,     0,     3],
       [14432,     0,     1],
       [15088,     0,     2],
       [15744,     0,     1],
       [16400,     0,     2]])

In [65]:
data = raw_data2.get_data()
print(type(data))
data.shape

<class 'numpy.ndarray'>


(64, 16960)

In [9]:
epochs = mne.Epochs(raw_data,
                    events,
                    event_id={'T1' : 2, 'T2' : 3},
                    tmin=0.0,
                    tmax=4.0,
                    baseline=(0.0, 0.0),
                    preload=True)

Not setting metadata
15 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 15 events and 641 original time points ...
0 bad epochs dropped


In [10]:
epochs

<Epochs | 15 events (all good), 0 – 4 s (baseline 0 – 0 s), ~4.8 MB, data loaded,
 'T1': 7
 'T2': 8>

In [71]:
import numpy as np
import mne
#TODO : make combined EEG MI

#All run : Event 0 corresponds to rest

#3,4,7,8,11,12 run : Event 1 corresponds left fist 
#5,6,9,10,13,14 run : Event 1 corresponds both fists

#3,4,7,8,11 run : Event 2 corresponds right fist 
#5,6,9,10,13 run : Event 2 corresponds both feet

#first event 672 => 672/160hz = 4.2sec events

run1 = [3,4,7,8,11,12]
run2 = [5,6,9,10,13,14]

hz = 160
channel = 64
sec = 4

left_fist_li = []
right_fist_li = []
both_fist_li = []
both_feet_li = []

for i in range(1, 110):
    
    subject = ""

    if(i>=100):
        subject = "S" + str(i)
    elif(i>=10):
        subject = "S0" + str(i)
    else:
        subject = "S00" + str(i)
    
    for j in range(3, 15):

        run = ""
        if(j<10):
            run = "R0" + str(j)
        else:
            run = "R" + str(j)

        url = '../Datasets/physio_mi/' + subject + '/' + subject + run + '.edf'
        
        #load
        raw = mne.io.read_raw_edf(url)
        data = raw.get_data()
        events, event_id = mne.events_from_annotations(raw)
        #print(events)
        #print(type(data), data.shape)
        
        #epoching

        for k in range(len(events)):
            
            tmp = data[:, events[k][0] : events[k][0] + (hz*sec)]

            if(np.shape(tmp)[0] != channel or np.shape(tmp)[1] != (hz * sec)): #(64, 640)
                continue

            #left fist
            if(j in run1 and events[k][-1] == 2):
                left_fist_li.append(tmp)
            elif(j in run1 and events[k][-1] == 3):
                right_fist_li.append(tmp)
            elif(j in run2 and events[k][-1] == 2):
                both_fist_li.append(tmp)
            elif(j in run2 and events[k][-1] == 3):
                both_feet_li.append(tmp)



left_fist = np.stack(left_fist_li, axis=0)
right_fist = np.stack(right_fist_li, axis=0)
both_fist = np.stack(both_fist_li, axis=0)
both_feet = np.stack(both_feet_li, axis=0)

Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S001/S001R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]


/tmp/ipykernel_683948/2420841302.py:49: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(url)
/tmp/ipykernel_683948/2420841302.py:49: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(url)
/tmp/ipykernel_683948/2420841302.py:49: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(url)
/tmp/ipykernel_683948/2420841302.py:49: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(url)
/tmp/ipykernel_683948/2420841302.py:49: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(url)
/tmp/ipykernel_683948/2420841302.py:49: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(url)
/tmp/ipykernel_683948/2420841302.py:49: RuntimeWarning: Li

Creating raw.info structure...
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S101/S101R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S101/S101R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S101/S101R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Extracting EDF parameters from /home/ys_kim/github/Datasets/physio_mi/S101/S101R08.edf...
EDF file detected
Setting channel info structure...


In [72]:
print(left_fist.shape)
print(right_fist.shape)
print(both_fist.shape)
print(both_feet.shape)

(4950, 64, 640)
(4894, 64, 640)
(4905, 64, 640)
(4924, 64, 640)


In [79]:
sources = np.concatenate([left_fist, right_fist, both_fist, both_feet], axis=0)
sources.shape

(19673, 64, 640)

In [75]:
target = [0] * 4950 + [1] * 4894 + [2] * 4905 + [3] * 4924
len(target)

19673

In [ ]:
np.save('../Datasets/physio_mi/sources.npy', sources)
np.save('../Datasets/physio_mi/target.npy', target)